## CSE482 - Project EDA
### Jack Nugent
Project Information can be accessed [here](https://www.cse.msu.edu/~wenhongz/cse482_22fall/project/Topics.pdf).

---
### Loading Tweet Data

- Want to pull 6-8 weeks worth of data
- Clarify if timeframe or just x number of tweets
- Tweets should have geolocation
- Tweets should have retweets removed

---
#### Stream of Tweets

In [2]:
import tweepy
from tweepy import OAuthHandler
from tweepy import API

# Replace with your Keys
C_KEY = None
C_SECRET = None
A_TOKEN_KEY = None
A_TOKEN_SECRET = None
BEARER = None

In [7]:
import time
from tweepy import OAuthHandler
from tweepy import Stream

class MyStreamingClient(tweepy.StreamingClient):
    def __init__(self, bearer_token, time_limit=20):
        self.start_time = time.time()
        self.limit = time_limit
        self.outFile = open('output.json', 'w')
        super(MyStreamingClient, self).__init__(bearer_token)
        
    def on_data(self, data):
        if (time.time() - self.start_time) < self.limit:
            self.outFile.write(data.decode("utf-8"))
            self.outFile.write('\n')
            return True
        else:
            self.outFile.close()
            self.disconnect()
            return False

    def on_error(self, status):
        print(status)

# Function to remove all rules from an existing stream_client
def remove_rules(stream_client):
    rules = list(stream_client.get_rules())[0]
    if rules != None:
        for rule in rules:
            id_ = rule.id
            stream_client.delete_rules(id_)
        
s_c = MyStreamingClient(bearer_token=BEARER)
remove_rules(s_c)

In [8]:
s_c.add_rules(tweepy.StreamRule('"Musk"'))
s_c.add_rules(tweepy.StreamRule('"Elon Musk"'))

s_c.get_rules()

Response(data=[StreamRule(value='"Musk"', tag=None, id='1593998056289992705'), StreamRule(value='"Elon Musk"', tag=None, id='1593998057011433474')], includes={}, errors=[], meta={'sent': '2022-11-19T16:02:06.640Z', 'result_count': 2})

In [9]:
s_c.filter(tweet_fields="entities")

Stream connection closed by Twitter


In [31]:
import json
f = open('output.json')

data_json = [json.loads(line) for line in f]
f.close()

In [32]:
for tweet in data_json:
    print(tweet)
    print()

{'data': {'edit_history_tweet_ids': ['1593783912911675393'], 'entities': {'annotations': [{'start': 27, 'end': 42, 'probability': 0.918, 'type': 'Person', 'normalized_text': 'Elizabeth Holmes'}, {'start': 50, 'end': 57, 'probability': 0.5068, 'type': 'Person', 'normalized_text': 'Theranos'}, {'start': 99, 'end': 109, 'probability': 0.8742, 'type': 'Person', 'normalized_text': 'Sam Bankman'}], 'mentions': [{'start': 3, 'end': 11, 'username': 'lindyli', 'id': '270132611'}]}, 'id': '1593783912911675393', 'text': 'RT @lindyli: We worshipped Elizabeth Holmes &amp; Theranos. Turns out she’s a fraud\n\nWe worshipped Sam Bankman-Fried &amp; FTX. Turns out he’s a fr…'}, 'matching_rules': [{'id': '1593783629540339713', 'tag': ''}]}

{'data': {'edit_history_tweet_ids': ['1593783917366108161'], 'entities': {'mentions': [{'start': 3, 'end': 18, 'username': 'diegowainstein', 'id': '135629267'}]}, 'id': '1593783917366108161', 'text': 'RT @diegowainstein: Elon Musk igualó a Macri.\nEs la segunda perso

In [33]:
for tweet in data_json:
    print(tweet['data']['text'])
    print()

RT @lindyli: We worshipped Elizabeth Holmes &amp; Theranos. Turns out she’s a fraud

We worshipped Sam Bankman-Fried &amp; FTX. Turns out he’s a fr…

RT @diegowainstein: Elon Musk igualó a Macri.
Es la segunda persona que despilfarró 45 mil millones de dólares, en pocos días.
Sólo que Elo…

RT @EdRaposo_: Guga Noblat, o adolescente de 40 anos que depende dos amigos do papai pra arrumar emprego, chamou Elon Musk, multibilionário…

RT @louieponto: elon musk vai fechar o twitter pra ninguém mais compartilhar essa imagem https://t.co/OGs9McTRFs

Elon Musk on Friday asked any remaining Twitter employees who write software code to report to the 10th floor of the office in San Francisco by early afternoon, according to an email reviewed by Reuters. https://t.co/r4CiTvDyzz

cade meu engajamento, elon musk?

@lindyli Who is we? Sneaky to join Elons names with caught crooks.

@ChanahsCreation https://t.co/h6Zy19bA9S

@ThePatriotOasis Yep

@victoryengin65 @KariLake @dcexaminer Musk

Elon Musk eu 

---

#### Number of Tweets

Use [this](https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-recent) for v2 API?

Tweepy Client
- `tweet.fields`
    - geo, created_at
- `start_time` param:
    - ISO8601 Date format for the 6-8 weeks of data.

In [84]:
# Number of Tweets:
auth = tweepy.OAuthHandler(C_KEY, C_SECRET)
auth.set_access_token(A_TOKEN_KEY, A_TOKEN_SECRET)
api = tweepy.API(auth)

def num_tweet(keyword='michigan'):
    # Filter Retweets
    keyword = keyword + ' -filter:retweets'
    posts = api.search_tweets(q=keyword,geocode="39.8283,-98.5795,5000km",until='2022-11-19', lang='en', count=2000)
    #for tweet in posts:
    #    print("* "+str(tweet.text.encode("utf-8")))
    print("{} Tweets pulled.".format(len(posts)))
    return posts

In [85]:
data = num_tweet('musk')

100 Tweets pulled.


In [86]:
data[0]._json['created_at'], data[-1]._json['created_at']

('Fri Nov 18 23:59:56 +0000 2022', 'Fri Nov 18 23:39:35 +0000 2022')

#### Fields of Tweet 
https://www.geeksforgeeks.org/python-status-object-in-tweepy/ 

In [87]:
for i in range(len(data)):
    #print((tweet.user.location).split(','))
    print(i+1, ':', data[i].text)
    print(data[i]._json['created_at'])
    print('---------------------------------------------')

1 : Elon Musk calls for 'anyone' left at Twitter who can write software to meet him, or fly in, to help him 'better und… https://t.co/oLoQnR9Qba
Fri Nov 18 23:59:56 +0000 2022
---------------------------------------------
2 : @lindyli All hail Lord Musk.
Fri Nov 18 23:59:49 +0000 2022
---------------------------------------------
3 : Elon Musk as the Nero of Twitter. That fits.
Fri Nov 18 23:59:48 +0000 2022
---------------------------------------------
4 : About Media &amp; Journalism: I Was the Head of Trust and Safety at Twitter. This Is What Could Become of It.… https://t.co/aoAEw1oA4J
Fri Nov 18 23:59:15 +0000 2022
---------------------------------------------
5 : Looks like Twitter is slowly dying. A lot of people on YouTube are making videos about Twitter shutting down. If th… https://t.co/x9HfYYuK5G
Fri Nov 18 23:58:58 +0000 2022
---------------------------------------------
6 : Unknown for certain but next week's payroll likely already approved and in automated process... but 

In [160]:
for i in range(len(data)):
    if data[i].place != None:
        print(i+1, data[i].place.full_name)
        print('Coords',data[i].place.bounding_box.coordinates)
        print()

4 Burke, VA
Coords [[[-77.327307, 38.745249], [-77.227003, 38.745249], [-77.227003, 38.841994], [-77.327307, 38.841994]]]

30 South Bend, IN
Coords [[[-86.385306, 41.598756], [-86.19642, 41.598756], [-86.19642, 41.760555], [-86.385306, 41.760555]]]

35 Staunton, VA
Coords [[[-79.113516, 38.121322], [-79.012337, 38.121322], [-79.012337, 38.190201], [-79.113516, 38.190201]]]

42 Forward Design Build Remodel
Coords [[[-83.85063934326172, 42.2890739440918], [-83.85063934326172, 42.2890739440918], [-83.85063934326172, 42.2890739440918], [-83.85063934326172, 42.2890739440918]]]

57 Ann Arbor, MI
Coords [[[-83.800262, 42.215087], [-83.659681, 42.215087], [-83.659681, 42.326592], [-83.800262, 42.326592]]]

60 Troy, MI
Coords [[[-83.209206, 42.5337815], [-83.0868811, 42.5337815], [-83.0868811, 42.624224], [-83.209206, 42.624224]]]

74 Clare, MI
Coords [[[-84.802956, 43.810084], [-84.748378, 43.810084], [-84.748378, 43.856857], [-84.802956, 43.856857]]]

91 Detroit, MI
Coords [[[-83.288056, 42.2

---
### Client

In [88]:
from tweepy import Client

In [91]:
client = Client(bearer_token=BEARER, consumer_key=C_KEY, consumer_secret=C_SECRET, 
                access_token=A_TOKEN_KEY, access_token_secret=A_TOKEN_SECRET)



---
### Preprocess data


- Lowercase
- Stopword Removal
    - Remove `@__` user mentions
- Stemming
    - Lemmatize?
- Use NLTK

In [24]:
import nltk
nltk.download("wordnet")
from nltk.corpus import wordnet
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /home/nugentj3/nltk_data...


### Hand Label Data
- Compute number of "positive" words and number of "negative" words
    - Can find [here](https://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html).
- Compute ratio of "positivity". 
    - If pos / (pos+neg) > P, classify as Positive
    - If pos / (pos+neg) < N, classify as Negative
    - Else, unclassified. Will predict on.
    
   

---
### Train Classifier

- Naive Bayes
    - SMOTE oversampling if needed
    - oversample after TTS
- SVM


---
### Extra Stuff to Do
- Use the geolocation to group together tweets from the same “region” (city, county or state, depending on whichever granularity you prefer). 

- Aggregate the tweets for each region per day and create a time series that shows the net daily sentiment (#positive tweets - #negative tweets) in the region.